In [24]:
import os
import json
import yaml
import openai
import src.generate_listings as generate_listings
import src.vector_db as vector_db
import src.personalized_response as personalized_response
import src.app as app

In [25]:
# Load configuration
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [26]:
# Extract relevant settings from the config file
api_base = config["env_config"]["api_base"]["base1"]
embedding_model = config["env_config"]["model_name"]["text2"]
file_path = config["data"]["listing_file_path"]
results_file_path = config["data"]["results_file_path"]

In [27]:
# api_key for openai
api_key = os.environ["OPENAI_API_KEY"]
openai.api_base = api_base
openai.api_key = api_key

In [28]:
# genelrate listing
# due to the lack of gpu resources and time, images were not generetaed but the code should be able to handle any image generation. 
# feel free to uncomment generate_image function inside the generate_listings module to generate images. 

generate_listings.main()

Generated 20 listings and saved to data/real_estate_listings.json
First two listings:
[
    {
        "neighborhood": "Sunset Hills",
        "price": 950000,
        "bedrooms": 4,
        "bathrooms": 3,
        "square_footage": "3,500 sqft",
        "description": "Welcome to this stunning 4-bedroom, 3-bathroom home located in the prestigious Sunset Hills neighborhood. This elegant estate features a grand entrance foyer, spacious living room with a fireplace, formal dining room, and a gourmet kitchen with top-of-the-line appliances. The luxurious master suite boasts a spa-like bathroom and a walk-in closet. The expansive backyard offers a sparkling pool, outdoor kitchen, and lush landscaping, perfect for entertaining guests. Enjoy luxury living at its finest in this Sunset Hills masterpiece.",
        "neighborhood_description": "Sunset Hills is known for its upscale vibe and picturesque views of the sunset over the rolling hills. This exclusive neighborhood is home to top-rated sc

In [29]:
# Load the listings from the JSON file
listing_collection = vector_db.load_listings()

In [30]:
# print first 2 listings
print(json.dumps(listing_collection[:2], indent=4))

[
    {
        "neighborhood": "Sunset Hills",
        "price": 950000,
        "bedrooms": 4,
        "bathrooms": 3,
        "square_footage": "3,500 sqft",
        "description": "Welcome to this stunning 4-bedroom, 3-bathroom home located in the prestigious Sunset Hills neighborhood. This elegant estate features a grand entrance foyer, spacious living room with a fireplace, formal dining room, and a gourmet kitchen with top-of-the-line appliances. The luxurious master suite boasts a spa-like bathroom and a walk-in closet. The expansive backyard offers a sparkling pool, outdoor kitchen, and lush landscaping, perfect for entertaining guests. Enjoy luxury living at its finest in this Sunset Hills masterpiece.",
        "neighborhood_description": "Sunset Hills is known for its upscale vibe and picturesque views of the sunset over the rolling hills. This exclusive neighborhood is home to top-rated schools, fine dining restaurants, and boutique shops. Residents can enjoy outdoor activi

In [ ]:
# store listings in vector db
vector_db.main()

In [32]:
# get user response
response = personalized_response.get_user_responses()
print(response)

- I am interested in a quiet neighborhood with a strong sense of community.
- My budget is $600,000.
- I need a minimum of 4 bedrooms.
- I need a minimum of 3 bathrooms.
- I require a minimum of 2,000 square feet.
- I prefer a Tudor style home with unique architectural features.
- I need a large kitchen with updated appliances and a spacious backyard.
- Amenities that are important to me include a home theater and a designated home gym.


In [33]:
# get perosnalized search listing
search_listings = personalized_response.search_and_retrieve(response)
print(search_listings)

[Document(page_content='Green Oaks | 800000 | 3 bedrooms | 2 bathrooms | 2,000 sqft sqft | Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem. | Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.'), Document(page_content=

In [34]:
# get perosnalized description for 1 listing
personalized_description = personalized_response.main(response)
print(personalized_description)

Personalized results saved to ./data/personalized_results.json
None


In [35]:
# get perosnalized description for multiple listings
personalized_response.main(response, num_results = 3)

Personalized results saved to ./data/personalized_results.json


In [36]:
# Load the JSON file and extract 'personalized_description'
with open(results_file_path, 'r') as f:
        personalized_results = json.load(f)

In [38]:
for result in personalized_results:
    for idx, listing in enumerate(result['personalized_listings'], start=1):
        print(f"{idx}. {listing['personalized_description']}\n")

1. Personalized Listing:

Property Description: Welcome to your new 4-bedroom, 3-bathroom craftsman-style home in the Sunny Brooke neighborhood. This spacious home features a cozy fireplace, a large kitchen with granite countertops, a two-car garage, and a backyard deck for outdoor relaxation.

Neighborhood Description: Sunny Brooke is a close-knit community with scenic parks, hiking trails, and top-rated schools. Enjoy the tranquility of tree-lined streets with easy access to shopping and dining options. Located near major highways for a convenient commute.

Listing Price: $580,000

2. Personalized Listing:

Property Description: Welcome to this charming 3-bedroom, 2-bathroom cottage-style home in the family-friendly Orchard Grove neighborhood. This home features a spacious backyard, perfect for gardening or enjoying outdoor activities. The cozy fireplace, updated kitchen, and hardwood floors add to the home's character and charm.

Neighborhood Description: Orchard Grove is known for 

In [ ]:
# run the whole code from the main file including the app built using gradio
app.main()